https://hub.packtpub.com/generating-automated-image-captions-using-nlp-and-computer-vision-tutorial/

In [26]:
# Import TensorFlow and enable eager execution
import tensorflow as tf
tf.enable_eager_execution()

AttributeError: module 'tensorflow' has no attribute 'enable_eager_execution'

In [4]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

## Image and Caption Data Loading

In [21]:
annotation_file = os.path.abspath('cocoapi/annotations/captions_train2014.json')
PATH = os.path.abspath('/Users/bwilke/Desktop/images/train2014/') # way to big for my cloud storage, so point to local machine

In [12]:
# read the json annotation file
with open(annotation_file, 'r') as f:
    annotations = json.load(f)

In [ ]:
# storing the captions and the image name in vectors
all_captions = []
all_img_name_vector = []

# extract caption and image ID to form full path from raw JSON annotations
for annot in annotations['annotations']:
    caption = ' ' + annot['caption'] + ' '
    image_id = annot['image_id']
    full_coco_image_path = PATH + 'COCO_train2014_{}.jpg'.format(image_id)

# adds the path and corresponding caption to shared index across 2 lists
all_img_name_vector.append(full_coco_image_path)
all_captions.append(caption)

# shuffling the captions and image_names together; setting a random state
train_captions, img_name_vector = shuffle(all_captions, all_img_name_vector, random_state = 1)

# selecting the first 40000 captions/images from the shuffled sey by replacing with sliced set
num_examples = 40000
train_captions = train_captions[:num_examples]
img_name_vector = img_name_vector[:num_examples]

## Image Loader, CNN Model, and Image Feature Extraction

In [ ]:
def load_image(image_path):
    img = tf.read_file(image_path)                                   # gets image from path    
    img = tf.image.decode_jpeg(img, channels=3)                      # decodes JPG
    img = tf.image.resize_images(img, (299, 299))                    # resize
    img = tf.keras.applications.inception_v3.preprocess_input(img)   # preprocess for inceptionV3
    return img, image_path

In [ ]:
# initialize Inception-V3 with ImageNet weights
# this is our CNN where output layer is the last layer in Inception-V3

image_model = tf.keras.applications.InceptionV3(include_top = False, weights='imagenet')  # loads pretrained model
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

# image_features_extract_model is our deep CNN encoder, which is responsible for learning
# the features from the given image.
image_features_extract_model = tf.keras.Model(new_input, hidden_layer)


In [ ]:
encode_train = sorted(set(img_name_vector))                # sort unique entries in img_name_vector

# for each image path in encode_train, apply load_image(), returns tensor
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train).map(load_image).batch(16)

# Extract features
for img, path in image_dataset:
    batch_features = image_features_extract_model(img)           # forward feed the image tensor through the model
    # reshape the output
    batch_features = tf.reshape(batch_features, (batch_features.shape[0], -1, batch_features.shape[3]))
    
# save each output to disk 
for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())    

## Data Preparation for RNN Decoder

In [ ]:
# Helper func to find the maximum length of any caption in our dataset
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

# Performing tokenization on the top 5000 words from the vocabulary
top_k = 5000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k, 
oov_token="", 
filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')

# Converting text into sequence of numbers
tokenizer.fit_on_texts(train_captions)
train_seqs = tokenizer.texts_to_sequences(train_captions)

tokenizer.word_index = {key:value for key, value in tokenizer.word_index.items() if value # putting  token in the word2idx dictionary
tokenizer.word_index[tokenizer.oov_token] = top_k + 1
tokenizer.word_index[''] = 0

# creating the tokenized vectors
train_seqs = tokenizer.texts_to_sequences(train_captions)

# creating a reverse mapping (index -> word)
index_word = {value:key for key, value in tokenizer.word_index.items()}

# padding each vector to the max_length of the captions
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

# calculating the max_length used to store the attention weights
max_length = calc_max_length(train_seqs)

In [ ]:
# result is a list of lists containing our encoded captions
cap_vector[1]

In [ ]:
# split 80/20 training and validation, where images are X and captions are Y
img_name_train, img_name_val, cap_train, cap_val = train_test_split(img_name_vector, cap_vector, test_size = 0.2, random_state = 69)

In [ ]:
# Checking the sample counts
print ("No of Training Images:",len(img_name_train))
print ("No of Training Caption: ",len(cap_train))
print ("No of Training Images",len(img_name_val))
print ("No of Training Caption:",len(cap_val))

## Setting Up Data Pipeline

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
# Defining parameters
BATCH_SIZE = 64
BUFFER_SIZE = 1000
embedding_dim = 256
units = 512
vocab_size = len(tokenizer.word_index)

In [ ]:
# shape of the vector extracted from Inception-V3 is (64, 2048)
# these two variables represent that
features_shape = 2048
attention_features_shape = 64

# loading the numpy files 
def map_func(img_name, cap):
    img_tensor = np.load(img_name.decode('utf-8')+'.npy')
    return img_tensor, cap

#We use the from_tensor_slices to load the raw data and transform them into the tensors
dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))

# Using the map() to load the numpy files in parallel
# NOTE: Make sure to set num_parallel_calls to the number of CPU cores you have
# https://www.tensorflow.org/api_docs/python/tf/py_func
dataset = dataset.map(lambda item1, item2: tf.py_func(map_func, [item1, item2], [tf.float32, tf.int32]), num_parallel_calls=8)

# shuffling and batching
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(1)

## Defining the Captioning Model

Features extracted from lower convolutional layer of Inception-v3 is (8,8,2048)

This is unrolled to (64,2048)

Then passed to single dense layer of CNN encoder, RNN attends over image to predict next word

In [ ]:
def gru(units):
    if tf.test.is_gpu_available():
        return tf.keras.layers.CuDNNGRU(units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
    else:
        return tf.keras.layers.GRU(units, return_sequences=True, return_state=True, recurrent_activation='sigmoid', recurrent_initializer='glorot_uniform')

## Attention

Implement the attention mechanism known as Bahdanau attention.

Takes the features from the CNN encoder of a shape of (batch_size, 64, embedding_dim).

This attention mechanism will return the context vector and the attention weights over the time axis.

In [ ]:
class BahdanauAttention(tf.keras.Model):
    
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
    
    def call(self, features, hidden): # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        # score shape == (batch_size, 64, hidden_size)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

        # attention_weights shape == (batch_size, 64, 1)
        # we get 1 at the last axis because we are applying score to self.V
        attention_weights = tf.nn.softmax(self.V(score), axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)
    
        return context_vector, attention_weights

## Training the Captioning Model

First, extract features we saved to .npy files and pass each through CNN encoder.

Then encoder output, hidden state(init to 0), and the decoder input (which is start token) are passed to decoder.

Decoder returns predictions and the decoder hidden state.

Decoder hidden state is passed back to the model and predictions are used to calculate loss.

Teacher forcing technique is used to decide next input to decoder.

Finally, calculate gradient and apply it the optimizer and back prop.

In [ ]:
EPOCHS = 20
loss_plot = []

In [ ]:
for epoch in range(EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
        loss = 0

        # initializing the hidden state for each batch because the captions are not related from image to image
        hidden = decoder.reset_state(batch_size=target.shape[0])
        dec_input = tf.expand_dims([tokenizer.word_index['']] * BATCH_SIZE, 1)

        with tf.GradientTape() as tape:
            features = encoder(img_tensor)

            for i in range(1, target.shape[1]): # passing the features through the decoder
                predictions, hidden, _ = decoder(dec_input, features, hidden)
                loss += loss_function(target[:, i], predictions)
                dec_input = tf.expand_dims(target[:, i], 1)         # using teacher forcing
                total_loss += (loss / int(target.shape[1]))

        variables = encoder.variables + decoder.variables

        gradients = tape.gradient(loss, variables)

        optimizer.apply_gradients(zip(gradients, variables), tf.train.get_or_create_global_step())

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, loss.numpy() / int(target.shape[1])))

    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / len(cap_vector))

    print ('Epoch {} Loss {:.6f}'.format(epoch + 1, total_loss/len(cap_vector)))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [ ]:
# 